In [1]:
import os
import pandas as pd
from bs4 import BeautifulSoup
from io import StringIO

In [2]:
SCORE_DIR = "data/scores"
SEASONS = list(range(2020,2024))
SEASONS

[2020, 2021, 2022, 2023]

In [3]:
box_scores = os.listdir(SCORE_DIR)

In [4]:
box_scores.sort()
len(box_scores)

6358

In [5]:
box_scores = [os.path.join(SCORE_DIR, f) for f in box_scores if f.endswith("html")]
print(box_scores)
box_scores_list = []
for i in SEASONS:
    box_scores_i = [b for b in box_scores if b.startswith(f"data/scores/{i}")]
    box_scores_list.extend(box_scores_i)
    

['data/scores/201602010ATL.html', 'data/scores/201602010BRK.html', 'data/scores/201602010DEN.html', 'data/scores/201602010IND.html', 'data/scores/201602010NOP.html', 'data/scores/201602010OKC.html', 'data/scores/201602010SAC.html', 'data/scores/201602010SAS.html', 'data/scores/201602010UTA.html', 'data/scores/201602020HOU.html', 'data/scores/201602020LAL.html', 'data/scores/201602020NYK.html', 'data/scores/201602020PHO.html', 'data/scores/201602020POR.html', 'data/scores/201602030BOS.html', 'data/scores/201602030BRK.html', 'data/scores/201602030CHO.html', 'data/scores/201602030DAL.html', 'data/scores/201602030LAC.html', 'data/scores/201602030OKC.html', 'data/scores/201602030PHI.html', 'data/scores/201602030SAC.html', 'data/scores/201602030SAS.html', 'data/scores/201602030UTA.html', 'data/scores/201602030WAS.html', 'data/scores/201602040DET.html', 'data/scores/201602040NOP.html', 'data/scores/201602040PHO.html', 'data/scores/201602040POR.html', 'data/scores/201602050ATL.html', 'data/sco

In [6]:
def parse_html(box_scores):
    with open(box_scores) as f:
        html = f.read()
        
    soup = BeautifulSoup(html)
    [s.decompose() for s in soup.select("tr.over_header")]
    [s.decompose() for s in soup.select("tr.thead")]
    return soup

In [7]:
def read_line_score(soup):
    line_score = pd.read_html(StringIO(str(soup)),attrs={"id": "line_score"})[0]
    
    # Adjust column names
    cols = list(line_score.columns)
    cols[0] = "team"
    cols[-1] = "total"
    line_score.columns = cols
    
    # Grab useful data
    line_score = line_score[["team", "total"]]
    return line_score

In [8]:
def read_stats(soup, team, stat):
    
    # stat = "basic" for basic stats, stat = "advanced" for advanced stats
    df = pd.read_html(StringIO(str(soup)), attrs={"id":f"box-{team}-game-{stat}"}, index_col=0)[0]
    df = df.apply(pd.to_numeric, errors = "coerce")
    return df

In [9]:
def read_season_info(soup):
    nav = soup.select("#bottom_nav_container")[0]
    href = [a["href"] for a in nav.find_all("a")]
    season = os.path.basename(href[1]).split("_")[0]
    return season

In [10]:
base_cols = None
games = []

for box_score in box_scores_list:
    soup = parse_html(box_score)
    line_score = read_line_score(soup)
    teams = list(line_score["team"])

    summaries = []
    for team in teams:
        
        # Read basic and advanced stats
        basic = read_stats(soup, team, "basic")
        advanced = read_stats(soup, team, "advanced")
        
        # Create useful data
        totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]])
        totals.index = totals.index.str.lower()
        
        maxes = pd.concat([basic.iloc[:-1].max(),advanced.iloc[:-1].max()])
        maxes.index = maxes.index.str.lower() + "_max"
        
        summary = pd.concat([totals, maxes])
        
        if base_cols is None:
            base_cols = list(summary.index.drop_duplicates(keep="first"))
            base_cols = [b for b in base_cols if "bpm" not in b] # Remove "bpm" column
            
        summary = summary[base_cols]
        
        summaries.append(summary)
    
    summary = pd.concat(summaries, axis=1).T


    # Combine all needed data in a game
    game = pd.concat([summary, line_score], axis=1)
    game["home"] = [0,1]

    # Create stats for opponent4
    game_opp = game.iloc[::-1].reset_index()
    game_opp.columns += "_opp"

    full_game = pd.concat([game, game_opp], axis=1)

    # Specific which season is the game
    full_game["season"] = read_season_info(soup)

    # Specific the data of the game
    full_game["date"] = os.path.basename(box_score)[:8]
    full_game["date"] = pd.to_datetime(full_game["date"], format = "%Y%m%d")

    full_game["won"] = full_game["total"] > full_game["total_opp"]
    games.append(full_game)
    
    if len(games) %50 ==0:
        print(f"Progress: {len(games)}/{len(box_scores_list)}")


Progress: 50/4449
Progress: 100/4449
Progress: 150/4449
Progress: 200/4449
Progress: 250/4449
Progress: 300/4449
Progress: 350/4449
Progress: 400/4449
Progress: 450/4449
Progress: 500/4449
Progress: 550/4449
Progress: 600/4449
Progress: 650/4449
Progress: 700/4449
Progress: 750/4449
Progress: 800/4449
Progress: 850/4449
Progress: 900/4449
Progress: 950/4449
Progress: 1000/4449
Progress: 1050/4449
Progress: 1100/4449
Progress: 1150/4449
Progress: 1200/4449
Progress: 1250/4449
Progress: 1300/4449
Progress: 1350/4449
Progress: 1400/4449
Progress: 1450/4449
Progress: 1500/4449
Progress: 1550/4449
Progress: 1600/4449
Progress: 1650/4449
Progress: 1700/4449
Progress: 1750/4449
Progress: 1800/4449
Progress: 1850/4449
Progress: 1900/4449
Progress: 1950/4449
Progress: 2000/4449
Progress: 2050/4449
Progress: 2100/4449
Progress: 2150/4449
Progress: 2200/4449
Progress: 2250/4449
Progress: 2300/4449
Progress: 2350/4449
Progress: 2400/4449
Progress: 2450/4449
Progress: 2500/4449
Progress: 2550/4449


In [11]:
games_df = pd.concat(games, ignore_index = True)

In [12]:
games_df

,mp,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
0,240.0,240.0,42.0,86.0,0.488,6.0,23.0,0.261,17.0,23.0,...,20.5,33.7,146.0,117.0,LAL,117,1,2020,2020-01-01,False
1,240.0,240.0,43.0,90.0,0.478,7.0,28.0,0.250,24.0,34.0,...,100.0,39.1,204.0,125.0,PHO,107,0,2020,2020-01-01,True
2,240.0,240.0,34.0,91.0,0.374,15.0,46.0,0.326,21.0,28.0,...,33.8,36.8,176.0,106.0,MIL,106,1,2020,2020-01-01,False
3,240.0,240.0,40.0,94.0,0.426,9.0,36.0,0.250,17.0,26.0,...,17.0,38.4,211.0,103.0,MIN,104,0,2020,2020-01-01,True
4,240.0,240.0,37.0,94.0,0.394,10.0,32.0,0.313,9.0,15.0,...,19.4,28.6,230.0,103.0,NYK,117,1,2020,2020-01-01,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8893,240.0,240.0,34.0,92.0,0.370,11.0,35.0,0.314,15.0,19.0,...,21.5,33.4,202.0,113.0,DEN,109,0,2023,2023-06-07,False
8894,240.0,240.0,39.0,79.0,0.494,14.0,28.0,0.500,16.0,21.0,...,25.2,30.3,223.0,127.0,MIA,95,1,2023,2023-06-09,True
8895,240.0,240.0,35.0,78.0,0.449,8.0,25.0,0.320,17.0,20.0,...,50.0,32.6,164.0,114.0,DEN,108,0,2023,2023-06-09,False
8896,240.0,240.0,33.0,96.0,0.344,9.0,35.0,0.257,14.0,16.0,...,28.6,26.4,202.0,104.0,DEN,94,1,2023,2023-06-12,False


In [13]:
games_df.to_csv("nba_games2.csv")

Have problem with column 
gmsc | +/-, mp_max